<a href="https://colab.research.google.com/github/lealec1234/Event_camera_object_detection_tracking/blob/main/Graph_spectral_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Spectral clustering

In [ ]:
!dir #check current directory make sure to be in the dataset directory

drive  sample_data


In [ ]:
file = dtable.fread('/content/drive/MyDrive/honours project/Event_Camera/dataset_dir/DAVIS 240C Dataset/events.txt', sep = ' ').to_pandas()
file.columns = ['timestamp', 'x', 'y', 'polarity']
file['polarity'] *= 1
ts = np.array(file['timestamp'].values)
x = np.array(file['x'].values)
y = np.array(file['y'].values)
pol = np.array(file['polarity'].values)

print(ts.shape)
print(x.shape)
print(y.shape)

(23126288,)
(23126288,)
(23126288,)


In [ ]:
np.unique(ts).shape

(18119595,)

In [ ]:
#function to cluster the event camera datasets
import os
import matplotlib.pyplot as plt
import random
import h5py
import numpy as np
import warnings
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
print('reading davis dataset...')

#davis parameters
file = dtable.fread('/content/drive/MyDrive/TRM/DAVIS 240C Dataset/events.txt', sep = ' ').to_pandas()
file.columns = ['timestamp', 'x', 'y', 'polarity']
file['polarity'] *= 1
ts = np.array(file['timestamp'].values)
x = np.array(file['x'].values)
y = np.array(file['y'].values)
pol = np.array(file['polarity'].values)

warnings.filterwarnings('ignore', '.*Graph is not fully connected*')
ALL = len(pol)
NEIGHBORS = 30
print(str(ALL)+' events in dataset...')
seg = 1000
while seg >= 1000:
    print('dividing the sequence into '+str(seg)+' segments...')
    X = ALL//seg
    print('each segment has '+str(X)+' events, out of which '+str(X//4)+' events will be selected...')
    for sl_no in range(0,10):

        print('segment no: '+str(sl_no+1))
        selected_events = []
        for i in range(0,ALL)[sl_no*X:sl_no*X+X]:
            selected_events.append([y[i], x[i], ts[i]*0.0001, pol[i]*0])
        selected_events = np.asarray(selected_events)

        print('removing noise...')
        cleaned_events = IsolationForest(random_state=0, n_jobs=-1, contamination=0.05).fit(selected_events)
        unwanted_events = cleaned_events.predict(selected_events)
        selected_events_cleaned = selected_events[np.where(unwanted_events == 1, True, False)]

        print('constructing graph...')
        adMat_cleaned = kneighbors_graph(selected_events_cleaned, n_neighbors=NEIGHBORS)

        print('finding optimal number of clusters...')
        max_score = -20
        opt_clusters = 2
        for CLUSTERS in range(2, 10):
            clustering = SpectralClustering(n_clusters=CLUSTERS, random_state=0,
                                            affinity='precomputed_nearest_neighbors',
                                            n_neighbors=NEIGHBORS, assign_labels='kmeans',
                                            n_jobs=-1).fit_predict(adMat_cleaned)
            curr_score = silhouette_score(selected_events_cleaned, clustering)
            if curr_score > max_score:
                max_score = curr_score
                opt_clusters = CLUSTERS

        print('clustering...')
        clustering_opt = SpectralClustering(n_clusters=opt_clusters, random_state=0,
                                            affinity='precomputed_nearest_neighbors',
                                            n_neighbors=NEIGHBORS, assign_labels='kmeans',
                                            n_jobs=-1).fit_predict(adMat_cleaned)

        print('saving files...')

        yy = str(sl_no)
        file_name = yy
        os.makedirs('results/davis240/'+str(seg)+'/selected_events', exist_ok=True)
        os.makedirs('results/davis240/'+str(seg)+'/clusters', exist_ok=True)
        np.save(os.path.join('results/davis240/'+str(seg)+'/selected_events', file_name + '.npy'), selected_events_cleaned)
        np.save(os.path.join('results/davis240/'+str(seg)+'/clusters', file_name + '.npy'), clustering_opt)
    break
print('done')

reading davis dataset...
23126288 events in dataset...
dividing the sequence into 1000 segments...
each segment has 23126 events, out of which 5781 events will be selected...
segment no: 1
removing noise...
constructing graph...
finding optimal number of clusters...
clustering...
saving files...
segment no: 2
removing noise...
constructing graph...
finding optimal number of clusters...
clustering...
saving files...
segment no: 3
removing noise...
constructing graph...
finding optimal number of clusters...
clustering...
saving files...
segment no: 4
removing noise...
constructing graph...
finding optimal number of clusters...
clustering...
saving files...
segment no: 5
removing noise...
constructing graph...
finding optimal number of clusters...
clustering...
saving files...
segment no: 6
removing noise...
constructing graph...
finding optimal number of clusters...
clustering...
saving files...
segment no: 7
removing noise...
constructing graph...
finding optimal number of clusters...
c